In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import json
import os
import time
import logging

In [7]:
BASE_URL = 'https://cn.reuters.com/news/archive/topic-cn-top-news?'

HEADERS = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/78.0.3904.97 Safari/537.36'
}

params_base = {
    'view': 'page',
    'page': '{!s}',
    'pageSize': 10
}

In [8]:
param = params_base
params['page'] = 1
url = BASE_URL + urlencode(params)
resp = requests.get(url, headers=HEADERS)

In [9]:
resp

<Response [200]>

In [11]:
html = BeautifulSoup(resp.text, 'lxml')

In [29]:
articles = html.select('div.news-headline-list article')

In [34]:
titles = [tag.select('h3.story-title')[0] for tag in articles]

In [39]:
titles

[<h3 class="story-title">
 								今明两日部分中国大陆往来香港列车临时停运--中国铁路官网</h3>, <h3 class="story-title">
 								亚股涨跌不一 市场对美中贸易协议的疑虑加重</h3>, <h3 class="story-title">
 								香港特首称必须“和平解决”理工大学校园事件 有信心警队能处理</h3>, <h3 class="story-title">
 								日本央行总裁称央行正在研究数字货币 但目前没有计划发行</h3>, <h3 class="story-title">
 								WTO会是特朗普贸易战的下一个受害者吗？</h3>, <h3 class="story-title">
 								焦点：美联储当前货币政策“适当” 将观望经济发展--联储梅斯特</h3>, <h3 class="story-title">
 								2020年美国经济增长前景改善 尽管存在贸易风险--房利美</h3>, <h3 class="story-title">
 								白宫暗示将支持临时拨款法案以避免政府关门</h3>, <h3 class="story-title">
 								美国众议院在调查特朗普是否在通俄门调查中说谎</h3>, <h3 class="story-title">
 								中国对达成贸易协议持悲观态度 因特朗普不愿取消关税--CNBC</h3>]

In [36]:
contents = [tag.select('div.story-content p')[0] for tag in articles]

In [43]:
class Page:

    _count = 0

    def __init__(self, titles:list=None, contents:list=None):
        self._data = dict()
        if titles and contents:
            for i, (t, c) in enumerate(zip(titles, contents)):
                self._data[f'article_{i}'] = dict(
                    title=t,
                    content=c
                )
        self.count = type(self)._count
        type(self)._count += 1
    
    @classmethod
    def reset_page_count(cls):
        cls._count = 0
    
    def __getitem__(self, index:int):
        if index < 0 or index > len(self._data):
            raise IndexError(f'Index "{index}" out of range')
        else:
            return self._data[f'article_{index}']
    
    def __str__(self):
        name = type(self).__name__
        return f'{name}_{self.count}(article_count={len(self._data)})'
    
    __repr__ = __str__

In [52]:
Page.reset_page_count()
p1 = Page([i.text.strip() for i in titles], 
          [i.text.strip() for i in contents])

In [53]:
p1

Page_0(article_count=10)

In [54]:
p1[0]

{'title': '今明两日部分中国大陆往来香港列车临时停运--中国铁路官网',
 'content': '路透北京11月19日 - 今明两日部分中国大陆往来香港列车临时停运。中国铁路武汉局集团发布临时停运公告称，周二（11月19日）北京西开Z97次运行至广州东终止，周三（11月20日）香港红磡开Z98次改为广州东始发，同时广州东—香港红磡Z97次停运、香港红磡—广州东Z98次停运。'}